#**JUPYTER NOTEBOOK FOR TRAINING THE NAIRA CLASSIFIER USED IN MONEYCOUNT**

**Author** 
: Oluwatobi Victor Ateniola

**Objective** : An implementation of a classifier to recognize images of the various denominations of the Naira banknote and also correctly classify the denominations. I built the classifier by modifying a pre-trained mobilenet neural network, I used the mobilenet architecture because of its relatively small size and accuracy. I modified the mobilenet architecure by changing the final sequential layer to classify the banknotes correctly. I made use of pytorch to create and train the model and then I converted the gradients of the trained model to the pytorch mobile format in order to easily integrate in a mobile application.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
dirr = "nairanet/"

In [0]:
import os
from torchvision import datasets
from PIL import Image
import numpy as np
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
## TODO: Specify data loaders

train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=45),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    ])


test_transforms = transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


train_data = datasets.ImageFolder(dirr + "naira_dataset/train" , transform=train_transforms)
test_data = datasets.ImageFolder(dirr + "naira_dataset/test" , transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data , batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data , batch_size=32, shuffle=False)

class_names = train_data.classes
loaders_transfer = {}

loaders_transfer["train"] = trainloader
loaders_transfer["valid"] = testloader

In [0]:
len(loaders_transfer["train"])

115

In [0]:
class_names

['05', '10', '100', '1000', '20', '200', '50', '500']

In [0]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()
use_cuda

True

In [0]:
import torchvision.models as models
import torch.nn as nn

## TODO: Specify model architecture 
nairanet = mobilenet = models.mobilenet_v2(pretrained=True)

for param in nairanet.parameters():
  param.requires_grad = False

nairanet.classifier = nn.Sequential(
                                nn.Dropout(0.4),
                                nn.Linear(1280, 512)
                                ,nn.ReLU(), 
                                nn.Dropout(0.3),
                                nn.Linear(512, 8)
                                )

if use_cuda:
    nairanet = nairanet.cuda()
  
# nairanet


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 94.7MB/s]


In [0]:
import torch.optim as optim

#Specifying the loss function and the optimizer
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(nairanet.classifier.parameters(), lr=0.04)

In [0]:
#Function for training the model
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            optimizer.zero_grad()
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            output = model.forward(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            loss = None
            with torch.no_grad():
              output = model.forward(data)
              loss = criterion(output, target)
            valid_loss += loss.item()
        # print training/validation statistics 
        train_loss = train_loss / len(loaders["train"])
        valid_loss = valid_loss / len(loaders["valid"])
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
            # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    # return trained model
    
    return model



In [0]:
#training the model
nairanet = train(50, loaders_transfer, nairanet, optimizer_transfer, criterion_transfer, use_cuda, dirr + 'model/nairanet.pt')

# load the model that got the best validation accuracy
nairanet.load_state_dict(torch.load(dirr + 'model/nairanet.pt'))

In [0]:
#Function for testing the model accuracy
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['valid']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
# test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

In [0]:
#Testing the model accuracy
test(loaders_transfer, nairanet , criterion_transfer, use_cuda)

In [0]:
##The block of code moves the code to cpu and then converts to pytorch's mobile format
nairanet.to("cpu")
nairanet.eval()
example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(nairanet, example)
traced_script_module.save(dirr + 'model/nairanetmobile.pt')